# Ejercicios Ejercicios PySpark clase 4

In [ ]:
!pip install pyspark==3.2.0 pyspark2pmml openscoring==0.5.0
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
!wget https://repo1.maven.org/maven2/org/jpmml/pmml-sparkml/2.2.0/pmml-sparkml-2.2.0.jar
!wget https://github.com/jpmml/jpmml-sparkml/releases/download/2.2.0/pmml-sparkml-example-executable-2.2.0.jar
!cp *.jar /usr/local/lib/python3.7/dist-packages/pyspark/jars
!wget https://github.com/openscoring/openscoring/releases/download/2.1.0/openscoring-server-executable-2.1.0.jar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 23.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805893 sha256=1a2047ad82fdfe7bc713af7ea1eb407db56aee8f653a7bf9dbed79cbbe75af49
  Stored in directory: /root/.cache/pip/wheels/32/97/d3/8b6d964c8700e4fbb561c71638a92ec55dac9be51eb5fea86d
  Created wheel for openscoring: filename=openscoring-0.5.0-py3-none-any.whl size=15444 sha256=9ef30276ac807a88129f9632b921933ecf398f90963eb2648b1f22990b3b5ba7
  Stored in directory: /root/.cache/pip/wheels/9e/dc/6d/f693595af7fc650ef112694173f372d1e9dcc692311cd9d6c1
  Created wheel for pyspark2pmml: filename=pyspark2pmml-0.5.1-py3-none-any.whl size=2398 sha256=692caea94844dce2eca9bd10f245eb13d5bc6699134ab0856a1c5f27bb55e4ca
  Stored in director

In [ ]:
import os
from openscoring import Openscoring

from pyspark2pmml import PMMLBuilder
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.types import StructType, DoubleType, StringType
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import (
    MinMaxScaler,
    VectorAssembler,
    OneHotEncoder,
    StringIndexer,
    IndexToString
)
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
jars = [
  'pmml-sparkml-example-executable-2.2.0.jar',
  'pmml-sparkml-2.2.0.jar',
  '/content/pmml-sparkml-example-executable-2.2.0.jar',
  '/content/pmml-sparkml-2.2.0.jar'
]

joined_jars = ",".join(jars)
os.environ['PYSPARK_SUBMIT_ARGS'] = \
  '--packages org.jpmml:pmml-sparkml:2.0.0 ' + \
  f'--master local[*] --jars {joined_jars} ' + \
  'pyspark-shell'

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .config('spark.jars', joined_jars) \
    .appName("PMML") \
    .getOrCreate()
sc = spark.sparkContext

## Filtrado y selección de datos en PySpark
Dado el siguiente DataFrame de PySpark que contiene información de alumnos junto a sus respectivas notas en varios cursos, aplique los procesos de filtrado y selección de datos para obtener únicamente el nombre, apellido y promedio de notas de aquellos alumnos cuya nota promedio sea igual o mayor a 7.


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("PySpark Class 4").getOrCreate()

data = [('Alicia', 'López', 6, 9, 7),
        ('Carlos', 'Rodríguez', 8, 6, 8),
        ('Diana', 'Díaz', 9, 10, 9),
        ('Fernando', 'Martínez', 4, 3, 5),
        ('Gabriela', 'Sánchez', 7, 9, 6)]

columns = ['Nombre', 'Apellido', 'Nota Curso 1', 'Nota Curso 2', 'Nota Curso 3']

dataFrame = spark.createDataFrame(data, schema=columns)

In [ ]:
from pyspark.sql.functions import col, round

# Calcular promedio de notas
dataFrame = dataFrame.withColumn("Promedio", round((col("Nota Curso 1") + col("Nota Curso 2") + col("Nota Curso 3")) / 3, 2))

# Filtrar alumnos con promedio >= 7
result = dataFrame.filter(col("Promedio") >= 7).select("Nombre", "Apellido", "Promedio")

# Mostrar resultado
result.show()

+--------+---------+--------+
|  Nombre| Apellido|Promedio|
+--------+---------+--------+
|  Alicia|    López|    7.33|
|  Carlos|Rodríguez|    7.33|
|   Diana|     Díaz|    9.33|
|Gabriela|  Sánchez|    7.33|
+--------+---------+--------+



## Agrupamiento de datos por columna específica
Dado el siguiente DataFrame de PySpark que contiene información de ventas de productos por categoría, aplique el proceso de agrupamiento y agregación de datos para obtener el total de ventas por categoría.


In [ ]:
sales_data = [('Electrónicos', 30),
              ('Hogar', 25),
              ('Deportes', 28),
              ('Electrónicos', 42),
              ('Hogar', 11),
              ('Deportes', 15)]

sales_columns = ['Categoría', 'Ventas']

sales_df = spark.createDataFrame(sales_data, schema=sales_columns)

In [ ]:
from pyspark.sql.functions import sum

# Agrupar datos por categoría y sumar ventas
result = sales_df.groupBy("Categoría").agg(sum("Ventas").alias("Total de Ventas"))

# Mostrar resultado
result.show()

+------------+---------------+
|   Categoría|Total de Ventas|
+------------+---------------+
|Electrónicos|             72|
|    Deportes|             43|
|       Hogar|             36|
+------------+---------------+



## Conversión de columna categórica a numérica
Utilice el siguiente DataFrame de PySpark que contiene información de personas y su género. Convierta la columna 'Género' a numérica, asignando 0 para 'Femenino' y 1 para 'Masculino'.

In [ ]:
personas_data = [('Alicia', 'Femenino'),
                 ('Carlos', 'Masculino'),
                 ('Blanca', 'Femenino'),
                 ('Mateo', 'Masculino')]

personas_columns = ['Nombre', 'Género']

personas_df = spark.createDataFrame(personas_data, schema=personas_columns)

In [ ]:
from pyspark.sql.functions import when

# Convertir columna 'Género' a numérica
result = personas_df.withColumn("Género Numérico", when(col("Género") == "Femenino", 0).otherwise(1))

# Mostrar resultado
result.show()

+------+---------+---------------+
|Nombre|   Género|Género Numérico|
+------+---------+---------------+
|Alicia| Femenino|              0|
|Carlos|Masculino|              1|
|Blanca| Femenino|              0|
| Mateo|Masculino|              1|
+------+---------+---------------+



##Utilizando el conjunto de datos de Iris:

*   Cargue el conjunto de datos iris.csv en un DataFrame de PySpark.

In [ ]:
iris_schema = StructType().add('sepal.length', DoubleType()) \
  .add('sepal.width', DoubleType()) \
  .add('petal.length', DoubleType()) \
  .add('petal.width', DoubleType()) \
  .add('variety', StringType())

# renaming columns to remove dot for better compatibility
iris_df = spark.read.format('csv') \
  .schema(iris_schema) \
  .option('header', 'true') \
  .load('iris.csv') \
  .select(
      col('`sepal.width`').alias('sepal_width'),
      col('`sepal.length`').alias('sepal_length'),
      col('`petal.width`').alias('petal_width'),
      col('`petal.length`').alias('petal_length'),
      col('variety')
    )
# iris_df.show()
iris_df.printSchema()

root
 |-- sepal_width: double (nullable = true)
 |-- sepal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- variety: string (nullable = true)



##Utilizando PySpark MLlib:   
*   Genere una tarea de regresión: Prediga petal_length basándose en las otras características (sepal_length, sepal_width, petal_width)
* Divida el conjunto de datos en conjuntos de entrenamiento y prueba.

1.   Entrene un modelo de regresión lineal para la tarea definida.
2.   Exporte este modelo de regresión en formato PMML.

In [ ]:
(trainingData, testData) = iris_df.randomSplit([0.7, 0.3])

assembler = VectorAssembler(
    inputCols=["sepal_length", "sepal_width", "petal_width"],
    outputCol="features")

lr = LinearRegression(featuresCol="features", labelCol="petal_length")
pipeline = Pipeline(stages=[assembler, lr])
model = pipeline.fit(trainingData)

# Hacer predicciones en los conjuntos de entrenamiento y prueba
train_predictions = model.transform(trainingData)
test_predictions = model.transform(testData)

# Evaluar el modelo en el conjunto de entrenamiento
train_evaluator = RegressionEvaluator(labelCol="petal_length", predictionCol="prediction", metricName="mse")
train_mse = train_evaluator.evaluate(train_predictions)
train_rmse = train_mse**0.5
train_accuracy = 1 - train_rmse
train_error = train_rmse

# Evaluar el modelo en el conjunto de prueba
test_evaluator = RegressionEvaluator(labelCol="petal_length", predictionCol="prediction", metricName="mse")
test_mse = test_evaluator.evaluate(test_predictions)
test_rmse = test_mse**0.5
test_accuracy = 1 - test_rmse
test_error = test_rmse

print(f"Training MSE: {train_mse}, Training RMSE: {train_rmse}, Training Accuracy: {train_accuracy}, Training Error: {train_error}")
print(f"Test MSE: {test_mse}, Test RMSE: {test_rmse}, Test Accuracy: {test_accuracy}, Test Error: {test_error}")

pmmlBuilder = PMMLBuilder(sc, trainingData, model)
pmmlBuilder.buildFile("LinearRegressionIris.pmml")

Training MSE: 0.09427025670694064, Training RMSE: 0.3070346180920657, Training Accuracy: 0.6929653819079342, Training Error: 0.3070346180920657
Test MSE: 0.11343443252795576, Test RMSE: 0.3368002858192905, Test Accuracy: 0.6631997141807096, Test Error: 0.3368002858192905


'/content/LinearRegressionIris.pmml'